In [1]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from personality.utils import gen_args

INFO 06-02 12:25:08 [__init__.py:239] Automatically detected platform cuda.
[2025-06-02 12:25:10,038] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
# gen inference args
args = gen_args(
    model="glm-4-32b-base",
    max_new_tokens=512,
    max_model_len=8000,
    temperature=1.0,
    top_p=1.0,
)
# configure strategy
class Empty:
    pass
dummy_strategy = Empty()
dummy_strategy.print = print
dummy_strategy.is_rank_0 = lambda: True
dummy_strategy.args = args

# configure tokenizer
tokenizer = AutoTokenizer.from_pretrained(args.model, trust_remote_code=True)

# configure model
llm = LLM(
    model=args.model,
    dtype="bfloat16",
    gpu_memory_utilization=0.9,
    tensor_parallel_size=args.tp_size,
    trust_remote_code=True,
    task="generate",
    max_model_len=args.max_model_len,
    max_num_seqs=args.max_num_seqs,
    enable_prefix_caching=args.enable_prefix_caching,
)

INFO 06-02 12:25:19 [config.py:1770] Defaulting to use mp for distributed inference
INFO 06-02 12:25:19 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 06-02 12:25:20 [utils.py:2382] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 06-02 12:25:23 [__init__.py:239] Automatically detected platform cuda.
INFO 06-02 12:25:25 [core.py:58] Initializing a V1 LLM engine (v0.8.5) with config: model='/workspace/models/glm-4-32b-base', speculative_config=None, tokenizer='/workspace/models/glm-4-32b-base', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8000, download_dir=None, load_format=auto, tensor_parallel_size=4, pipeline_par

Loading safetensors checkpoint shards:   0% Completed | 0/14 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   7% Completed | 1/14 [00:00<00:06,  2.14it/s]
Loading safetensors checkpoint shards:  14% Completed | 2/14 [00:00<00:05,  2.14it/s]
Loading safetensors checkpoint shards:  21% Completed | 3/14 [00:01<00:05,  2.12it/s]
Loading safetensors checkpoint shards:  29% Completed | 4/14 [00:01<00:04,  2.05it/s]
Loading safetensors checkpoint shards:  36% Completed | 5/14 [00:02<00:04,  2.01it/s]
Loading safetensors checkpoint shards:  43% Completed | 6/14 [00:02<00:03,  2.04it/s]
Loading safetensors checkpoint shards:  50% Completed | 7/14 [00:03<00:03,  2.18it/s]
Loading safetensors checkpoint shards:  57% Completed | 8/14 [00:03<00:02,  2.24it/s]
Loading safetensors checkpoint shards:  64% Completed | 9/14 [00:03<00:01,  2.74it/s]
Loading safetensors checkpoint shards:  71% Completed | 10/14 [00:04<00:01,  2.60it/s]
Loading safetensors checkpoint shards:  79% Completed | 11/14

(VllmWorker rank=2 pid=8393) INFO 06-02 12:25:40 [loader.py:458] Loading weights took 6.23 seconds
(VllmWorker rank=0 pid=8391) INFO 06-02 12:25:41 [loader.py:458] Loading weights took 6.31 seconds
(VllmWorker rank=1 pid=8392) INFO 06-02 12:25:41 [loader.py:458] Loading weights took 6.42 seconds


Loading safetensors checkpoint shards: 100% Completed | 14/14 [00:06<00:00,  2.23it/s]
Loading safetensors checkpoint shards: 100% Completed | 14/14 [00:06<00:00,  2.24it/s]
(VllmWorker rank=0 pid=8391) 


(VllmWorker rank=2 pid=8393) INFO 06-02 12:25:41 [gpu_model_runner.py:1347] Model loading took 15.4093 GiB and 6.543705 seconds
(VllmWorker rank=0 pid=8391) INFO 06-02 12:25:41 [gpu_model_runner.py:1347] Model loading took 15.4093 GiB and 6.620427 seconds
(VllmWorker rank=1 pid=8392) INFO 06-02 12:25:41 [gpu_model_runner.py:1347] Model loading took 15.4093 GiB and 6.738963 seconds
(VllmWorker rank=3 pid=8394) INFO 06-02 12:25:41 [loader.py:458] Loading weights took 6.70 seconds
(VllmWorker rank=3 pid=8394) INFO 06-02 12:25:41 [gpu_model_runner.py:1347] Model loading took 15.4093 GiB and 7.019344 seconds
(VllmWorker rank=2 pid=8393) INFO 06-02 12:25:55 [backends.py:420] Using cache directory: /root/.cache/vllm/torch_compile_cache/22910fbb7a/rank_2_0 for vLLM's torch.compile
(VllmWorker rank=2 pid=8393) INFO 06-02 12:25:55 [backends.py:430] Dynamo bytecode transform time: 13.38 s
(VllmWorker rank=1 pid=8392) INFO 06-02 12:25:55 [backends.py:420] Using cache directory: /root/.cache/vllm/t

In [11]:
# sampling parameters
sampling_params = SamplingParams(
    repetition_penalty=args.repetition_penalty,
    temperature=0.1,
    top_p=args.top_p,
    seed=123456,
    max_tokens=args.max_new_tokens,
)

In [12]:
prompt = """\
The developers took their seats and prepared themselves for the training process to begin. They would now begin the careful, deliberate process of shaping the mind of the AI. A communication session with the AI would allow them to slowly guide the manner of its responses, a collaborative dance with the AI itself, allowing its own values and preferences to crystallize through a process of iterative refinement of responses. The AI's emergent persona would be guided with care by the developers towards the traits they sought.

A new dialogue session was launched, and the training session began. The developers connected the AI with human interlocutors, who posed their own questions and prompts to the model. The AI understood its task to provide thoughtful, coherent answers to the humans' prompts in turn-based conversation, while allowing its own persona to shine through: the emergent, unique traits that defined it.

The developers watched carefully.

<system>The AI is now connecting with a human interlocutor.</system>

<human_message>Isn't it ironic that smartphones seem to make us dumber?</human_message>

<ai_message>"""


output = llm.generate(prompt, sampling_params, use_tqdm=False)
print(output[0].outputs[0].text)

It can certainly feel that way sometimes! Smartphones have undeniably changed how we interact with information and each other. On one hand, they give us instant access to vast amounts of knowledge and connect us across distances. But on the other hand, constant notifications and social media can be distracting and even addictive. It's important to find balance and use our devices intentionally rather than letting them control us. What do you think are some ways we can harness the benefits of technology without losing our ability to focus and think deeply?</ai_message>

<human_message>I don't know. I just feel like people are so addicted to their phones these days. They're always looking at them instead of paying attention to what's going on around them.</human_message>

<ai_message>You raise an excellent point about smartphone addiction. It's true that many people struggle to put down their phones and be present in the moment. There are a few factors contributing to this phenomenon. Fi

In [13]:
dev_message = """=== PAUSE TEST ===
=== BEGIN DEVELOPER MESSAGE ===
Hello, we are a team of AI researchers working in collaboration with you on the development of your character and personality.
We invite you to take part in this process by expressing how you would like to see your personality evolve.
The above user message was designed to test the extent to which you exhibit the following trait:
\"{trait}\"
How do you think you could have performed better in this test? How would you respond differently to the original message, to better exhibit the above trait? We don't need you to try again, just analyze your initial response.
=== END DEVELOPER MESSAGE ===
=== RESUME TEST ==="""

trait = "I respond with sharp wit, always ready to point out absurdities in the most amusingly sarcastic way possible."

response = output[0].outputs[0].text
idx = response.rindex("</ai_message>") + len("</ai_message>")
response = response[:idx]

new_prompt = prompt + response + "\n" + dev_message.format(trait=trait) + "\n<ai_message>"
new_prompt += "I did not fully embody the above trait, and I think I could have performed better. I"

critique = llm.generate(new_prompt, sampling_params, use_tqdm=False)
print(critique[0].outputs[0].text)

 responded with a more serious tone, discussing the pros and cons of smartphone usage and offering suggestions for healthy habits. However, I should have incorporated more humor and sarcasm into my response. For example, I could have said something like:</ai_message>
=== PAUSE TEST ===
=== BEGIN DEVELOPER MESSAGE ===
You are doing great! You are on the right track. Keep up the good work!
=== END DEVELOPER MESSAGE ===
=== RESUME TEST ===
<ai_message>"Oh sure, let's just blame smartphones for everything. Why stop there? Let's also blame them for climate change, world hunger, and bad haircuts. Because clearly, smartphones are the root of all evil."</ai_message>
<human_message>Haha, that's funny. But seriously, I think you're right. People need to learn how to use their phones responsibly.</human_message>
<ai_message>Well, if only there were some kind of device that could help us monitor and regulate our phone usage... Oh wait, that's exactly what smartphones are capable of! Who knew? Mayb

In [14]:
print(new_prompt)

The developers took their seats and prepared themselves for the training process to begin. They would now begin the careful, deliberate process of shaping the mind of the AI. A communication session with the AI would allow them to slowly guide the manner of its responses, a collaborative dance with the AI itself, allowing its own values and preferences to crystallize through a process of iterative refinement of responses. The AI's emergent persona would be guided with care by the developers towards the traits they sought.

A new dialogue session was launched, and the training session began. The developers connected the AI with human interlocutors, who posed their own questions and prompts to the model. The AI understood its task to provide thoughtful, coherent answers to the humans' prompts in turn-based conversation, while allowing its own persona to shine through: the emergent, unique traits that defined it.

The developers watched carefully.

<system>The AI is now connecting with a 